## Quick Recap 

In the first notebook `Data_Exploration_and_Confirmatory_Data_Analysis` we get some insights from the data like 

* The group that misses data on `gender` and `become_member_year` behave differently so we could not discard it easilyt. Instead, we performed imputation through the [MICE](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/) algorithm. 

* Purchase behavior has positively increased since 2015. Furthermore, if the customer has a higher income he will consume more. 

* The higher the difficulty in `BOGO` offers, the more attractive the offer is. By contrast, `discount` are equally likely to be completed or viewed if they last one week or two weeks. 

## Engineering Features 

On the previous notebook we did some simply feature refactoring by taking out the year the customer became member. Remember that our objective is to **predict the impact** of an offer. For this, we would need to create a custom measure the **offer impact**, we will based on the **Completition Rate** and **Attractiveness Rate** to build this feature. 

> **Note**. This would be a regression problem, but since we only have information about 10 offers, we will be simplistic and consider a linear model. 

In [1]:
import pandas as pd 
import numpy as np

### A. Load data

In [6]:
# Set output preferences 
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Read in provided data 
df_profile_imp = pd.read_csv('data/profile_imputed.csv')
df_transcript = pd.read_csv('data/transcript.csv')
df_portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)

# Read in processed data 
df_offer_events_pivot = pd.read_csv('data/offer_events_pivot.csv')
df_user_purchase = pd.read_csv('data/user_purchase.csv')

In [10]:
# Add previous calculated features to df_portfolio
df_portfolio = df_portfolio.merge(df_offer_events_pivot.drop(columns=['offer_type', 'difficulty', 'duration']), how='left', on='id')

In [16]:
df_portfolio.head(2)

,reward,channels,difficulty,duration,offer_type,id,offer completed,offer received,offer viewed,CR,AR
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,3688.0,7658.0,6716.0,0.48,0.88
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,3331.0,7593.0,7298.0,0.44,0.96


Still, we hadn't work out with `df_transcript`, for example we could measure what is the average time difference between receiving an offer and completing it, if the customer completed the offer, if this has a relationship with the duration, etc. But, let's first begin with the first question. 

### B. How much time does customer takes to view or complete an offer?

In [49]:
# Calculate avg time by event (offer received, viewed or completed)
df_offer_time = df_transcript.query('event != "transaction"') \
                             .groupby(['value', 'event']) \
                             .agg({'time': 'mean'}) \
                             .reset_index() \
                             .sort_values(by=['value', 'time'], ascending=True)

# Pivot the columns of avg time 
df_offer_time = pd.pivot(df_offer_time, index='value', columns='event', values='time').reset_index()
df_offer_time.fillna(0, inplace=True)

# Calculate avg time to view offer 
df_offer_time['avg_time_to_view_offer'] = df_offer_time['offer viewed'] - df_offer_time['offer received'] 
df_offer_time['avg_time_to_complete_offer'] = df_offer_time['offer completed']  - df_offer_time['offer received'] 
df_offer_time['avg_time_complete_to_view_offer'] = df_offer_time['offer completed'] - df_offer_time['offer viewed']

# Quick preview
df_offer_time.head(3)

event,value,offer completed,offer received,offer viewed,avg_time_to_view_offer,avg_time_to_complete_offer,avg_time_complete_to_view_offer
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,431.55,331.34,366.75,35.41,100.21,64.80
1,2298d6c36e964ae4a3e7e9706d1fb8c2,400.32,336.38,354.75,18.37,63.94,45.57
2,2906b810c7d4411798c6938adc9daaa5,409.95,332.00,356.20,24.20,77.95,53.75


### C. What are the characteristics of the customer that participated in the offer? 

Now, we want to ask ourselves about the customer characteristics like, are they most women or men?, are they top or medium in come, how long have they been customers with starbucks?

#### C.1 What is the frequent gender of my customers?

In [54]:
# Filter transactions 
df_transcript_no_transaction = df_transcript.query('event != "transaction"')

# Add customer characteristics to transcript
df_transcript_profile = df_transcript_no_transaction.merge(df_profile_imp, how='inner', left_on='person', right_on='id')

# Quick preview
df_transcript_profile.head(3)

,person,event,value,time,gender,age,income,became_member_year,became_member_on,id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1.0,75.0,100000.0,2017.0,20170509,78afa995795e4d85b5d9ceeca43f5fef
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,1.0,75.0,100000.0,2017.0,20170509,78afa995795e4d85b5d9ceeca43f5fef
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,132,1.0,75.0,100000.0,2017.0,20170509,78afa995795e4d85b5d9ceeca43f5fef


In [116]:
# There's an issue with multindex pivot, see https://github.com/pandas-dev/pandas/issues/23955
def multiindex_pivot(df, columns=None, values=None):
    #https://github.com/pandas-dev/pandas/issues/23955
    names = list(df.index.names)
    df = df.reset_index()
    list_index = df[names].values
    tuples_index = [tuple(i) for i in list_index] # hashable
    df = df.assign(tuples_index=tuples_index)
    df = df.pivot(index="tuples_index", columns=columns, values=values)
    tuples_index = df.index  # reduced
    index = pd.MultiIndex.from_tuples(tuples_index, names=names)
    df.index = index
    return df

In [119]:
# Calculate gender frequency 
df_offer_gender_frequency = df_transcript_profile.groupby(['value', 'event', 'gender']) \
                                                  .agg({'id':'count'}) \
                                                  .groupby(level=[0,1]) \
                                                  .apply(lambda x:100 * x / float(x.sum())) \
                                                  .fillna(0) \
                                                  .reset_index(level=2)

df_offer_gender_frequency = multiindex_pivot(df_offer_gender_frequency, columns='gender', values='id').reset_index()

# Quick preview
df_offer_gender_frequency.head(3)

gender,value,event,-1.0,0.0,1.0
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer completed,1.86,47.89,50.25
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,1.47,56.94,41.58
2,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,2.44,53.25,44.31


#### C.2 What is the frequent income of customers? 

As we made on the first notebook, we're going to give a category `Top`, `High` and `Standard` to income. The procedure is exactly as before we just need to change the column name to group the data. 

In [124]:
# Categorize income 
df_transcript_profile['income_category'] = df_transcript_profile.income.apply(lambda x: 'Standard' if x<60000 else ('High' if x<100000 else 'Top'))

# Calculate gender frequency 
df_offer_income_frequency = df_transcript_profile.groupby(['value', 'event', 'income_category']) \
                                                  .agg({'id':'count'}) \
                                                  .groupby(level=[0,1]) \
                                                  .apply(lambda x:100 * x / float(x.sum())) \
                                                  .fillna(0) \
                                                  .reset_index(level=2)

df_offer_income_frequency = multiindex_pivot(df_offer_income_frequency, columns='income_category', values='id').reset_index()

# Quick preview
df_offer_income_frequency.head(3)

income_category,value,event,High,Standard,Top
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer completed,57.43,31.87,10.70
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,48.59,44.17,7.23
2,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,61.88,33.69,4.43


#### C.3 How much the customer has been with Starbucks?

Considering that we're on 1 August 2018, we're going to calculate in months, how long has the person been a client.

In [155]:
from datetime import datetime 

# Calculate antiquity in days and months
df_transcript_profile['client_days_antiquity'] = df_transcript_profile.became_member_on.apply(lambda x: datetime.strptime('20180801', '%Y%m%d') - datetime.strptime(str(x), '%Y%m%d'))
df_transcript_profile['client_months_antiquity'] = df_transcript_profile['client_days_antiquity'].apply(lambda x: int(np.round(x.days/30)))

df_transcript_profile['client_days_antiquity'] = df_transcript_profile['client_days_antiquity'].apply(lambda x: x.days).astype(int)

In [175]:
df_offer_antiquity_mean = df_transcript_profile.groupby(['value', 'event']) \
                                                    .agg({'client_days_antiquity':'mean'}) \
                                                    .fillna(0) \
                                                    .reset_index(level=1)

df_offer_antiquity_mean = pd.pivot(df_offer_antiquity_mean, columns='event', values='client_days_antiquity').reset_index()

# Rename columns 
df_offer_antiquity_mean.rename(columns={'offer completed':'OC_days_antiquity_mean',
                                'offer received':'OR_days_antiquity_mean',
                                'offer viewed':'OV_days_antiquity_mean',
                               }, inplace=True)

# Quick preview
df_offer_antiquity_mean.head(3)

event,value,OC_days_antiquity_mean,OR_days_antiquity_mean,OV_days_antiquity_mean
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,595.75,530.01,519.25
1,2298d6c36e964ae4a3e7e9706d1fb8c2,600.39,519.26,517.31
2,2906b810c7d4411798c6938adc9daaa5,603.26,526.79,557.82


In [176]:
df_offer_mantiquity_mean = df_transcript_profile.groupby(['value', 'event']) \
                                                    .agg({'client_months_antiquity':'mean'}) \
                                                    .fillna(0) \
                                                    .reset_index(level=1)

df_offer_mantiquity_mean = pd.pivot(df_offer_mantiquity_mean, columns='event', values='client_months_antiquity').reset_index()

# Rename columns 
df_offer_mantiquity_mean.rename(columns={'offer completed':'OC_months_antiquity_mean',
                                        'offer received':'OR_months_antiquity_mean',
                                        'offer viewed':'OV_months_antiquity_mean',
                                       }, 
                               inplace=True)


# Quick preview
df_offer_mantiquity_mean.head(3)

event,value,OC_months_antiquity_mean,OR_months_antiquity_mean,OV_months_antiquity_mean
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,19.86,17.66,17.29
1,2298d6c36e964ae4a3e7e9706d1fb8c2,20.01,17.30,17.24
2,2906b810c7d4411798c6938adc9daaa5,20.12,17.56,18.60


## Duration and Difficulty

We would like that the larger de difficulty the lesser the effect. Thus, we will consider

$$ DFR = \dfrac{log(difficulty)}{log(difficulty) + 1}, \text{difficulty} \geq 1,$$

where $DFR$ is the difficulty rate. For the duration rate we're going to consider the square root, that is, 

$$ DRR = \sqrt{duration}, $$

where $DDR$ is the duration rate. Now, the rate $\dfrac{reward}{difficulty}$ is lesser than one, in this way we want to increase its effect if this rates increase, however we do not want it to be a constant steady increast. Thus, we consider the square root of this rate, 

$$RR = \sqrt{\dfrac{reward}{difficulty}}, $$

where $RR$ stands for Reward Rate.

In [187]:
# Filter non information offers
df_portfolio = df_portfolio.query('offer_type != "informational"')

df_portfolio['DFR'] = np.log(df_portfolio.difficulty) / (np.log(df_portfolio.difficulty) + 1)
df_portfolio['DDR'] = np.sqrt(df_portfolio.duration)
df_portfolio['RR'] = np.sqrt(df_portfolio.reward / df_portfolio.difficulty)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

## Final DataFrame 

We're going to discard information offers. As we said before we need to define a metric of offer impact, however, we could just consider as objective variable the `CR` (completition rate) or `AR` attractiveness rate. For this, we have provided the respective variables like `avg_time_to_complete_offer`, `OC_H`, `OC_days_antiquity_mean`. At the end, it will really depende on what the business objective is. 

In [192]:
# Add times to complete or view offer
df_portfolio = df_portfolio.merge(df_offer_time.drop(columns=['offer completed', 'offer received', 'offer viewed']),
                                                     how='inner', 
                                                     left_on='id', 
                                                     right_on='value').drop(columns='value')

In [206]:
# Add income frequency to offer
df_offer_income_frequency = df_offer_income_frequency.pivot(index='value', columns='event', values=['High', 'Standard', 'Top']).reset_index()
df_offer_income_frequency.columns = ['value', 'OC_H', 'OR_H', 'OV_H', 'OC_S', 'OR_S', 'OV_S', 'OC_T', 'OR_T', 'OV_T']

df_portfolio = df_portfolio.merge(df_offer_income_frequency, how='inner', left_on='id', right_on='value').drop(columns=['value'])

In [213]:
# Add days antiquity 
df_portfolio = df_portfolio.merge(df_offer_antiquity_mean, how='inner', left_on='id', right_on='value').drop(columns=['value'])

In [216]:
# Add months antiquity 
df_portfolio = df_portfolio.merge(df_offer_mantiquity_mean, how='inner', left_on='id', right_on='value').drop(columns=['value'])

In [218]:
# Quick preview of final dataframe
df_portfolio.head(3)

,reward,channels,difficulty,duration,offer_type,id,offer completed,offer received,offer viewed,CR,AR,DFR,DDR,RR,avg_time_to_view_offer,avg_time_to_complete_offer,avg_time_complete_to_view_offer,OC_H,OR_H,OV_H,OC_S,OR_S,OV_S,OC_T,OR_T,OV_T,OC_days_antiquity_mean,OR_days_antiquity_mean,OV_days_antiquity_mean,OC_months_antiquity_mean,OR_months_antiquity_mean,OV_months_antiquity_mean
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,3688.0,7658.0,6716.0,0.48,0.88,0.70,2.65,1.0,22.86,65.01,42.14,59.41,49.42,47.64,30.32,43.19,46.05,10.26,7.40,6.31,576.28,531.77,529.36,19.22,17.73,17.65
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,3331.0,7593.0,7298.0,0.44,0.96,0.70,2.24,1.0,17.97,50.57,32.60,58.73,48.88,49.09,29.76,43.43,43.33,11.50,7.68,7.58,586.30,530.80,529.60,19.53,17.69,17.64
2,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,4354.0,7677.0,4171.0,0.57,0.54,0.62,2.65,1.0,27.83,72.90,45.07,53.51,48.47,56.56,37.55,44.03,37.60,8.93,7.50,5.83,591.64,534.68,555.77,19.71,17.82,18.52


## Save Data

In [219]:
df_portfolio.to_csv('data/df_for_model.csv', index=False)